In [8]:
## second iteration


## demo for object tracking

import numpy as np
import cv2
import math
from IPython.display import clear_output
import time

avg_time = []

Conf_threshold = 0.4
NMS_threshold = 0.4
COLORS = [(0, 255, 0), (0, 0, 255), (255, 0, 0),
          (255, 255, 0), (255, 0, 255), (0, 255, 255)]
caps = cv2.VideoCapture('traffic_demo.mp4')

net = cv2.dnn.readNet('yolov4-opencv-python/yolov4-tiny.weights', 'yolov4-opencv-python/yolov4-tiny.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

# Initialize count
count = 0
center_points_prev_frame = []
center_points_cur_frame = []

tracking_objects = {}
tracking_objects2 = {}
track_id = 0

distance_thresh = 10 # how close objects can be between frames before counted as singular object

car_names = ['Sue', 'Billy','Bob','Alice','Jemma','Gertrude','Chad','Felicity','Grayson','Adam']

start_time = time.time()
while True:
    _, frame = caps.read()
    
    frame = cv2.resize(frame, (800, 400), fx = 0.1, fy = 0.1,interpolation = cv2.INTER_CUBIC)
    classes, scores, boxes = model.detect(frame, Conf_threshold, NMS_threshold)    
    
    
    for box in boxes: 
        (x,y,w,h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        center_points_cur_frame.append((cx,cy))
        # print("Frame N:", count, x, y, w, h )
        cv2.rectangle(frame,(x,y,),(x+w,y+h),(0,255,0),2)
        
    
    # cnt_pts_prv_frame = center_points_cur_frame.copy()
        
       # Only at the beginning we compare previous and current frame
    if count <= 1:
        for pt in center_points_cur_frame:
            for pt2 in center_points_prev_frame:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                if distance < distance_thresh:
                    tracking_objects[track_id] = pt
                    track_id += 1


                    
    else:
        tracking_objects_copy = tracking_objects.copy()
        center_points_cur_frame_copy = center_points_cur_frame.copy()
        for object_id, pt2 in tracking_objects_copy.items():
            object_exists = False
            for pt in center_points_cur_frame_copy:
                distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
                # Update IDs position
                if distance < distance_thresh:
                    # tracking_objects2[object_id] = tracking_objects[object_id]
                    tracking_objects[object_id] = pt
                    object_exists = True
                    if pt in center_points_cur_frame:
                        center_points_cur_frame.remove(pt)
                    continue
            # Remove IDs lost
            if not object_exists:
                tracking_objects.pop(object_id)
        # Add new IDs found
        for pt in center_points_cur_frame:
            tracking_objects[track_id] = pt
            track_id += 1
        
        
                    
                
    for object_id, pt in tracking_objects.items():
        cv2.circle(frame, pt,5,(0,0,255),-1)
        
        name_val = object_id % 10
        cv2.putText(frame,str(object_id),(pt[0], pt[1]-7),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
    
    # print(track_id, center_points_cur_frame,tracking_objects, cnt_pts_prv_frame
    # print("Tracking objects:")
    # print(tracking_objects)
                
        
        # cv2.circle(frame,pts, 2, (0,0,255),-1)
        
    #make copy of points
    clear_output(wait = True)
    print("Frame no: ", count)
    
    end_time = time.time()
    
    frame_time = end_time - start_time
    
    start_time = time.time()

    print("Time of frame(s): ", round(frame_time,4))
    avg_time.append(frame_time)
    
    average_frame_time = np.average(avg_time)
    print("Average time(s): ", round(average_frame_time,4))
    
    avg_fps = 1 / average_frame_time
    print("Average fps: ", round(avg_fps,4))
    
    cv2.imshow("Frame", frame)
    
    start_time = time.time()
    
    center_points_prev_frame = center_points_cur_frame.copy()
    
    
    center_points_cur_frame = []

    
    count += 1
              
    key = cv2.waitKey(1)
    if key == 27:
        break
        
caps.release()
cv2.destroyAllWindows()

Frame no:  67
Time of frame(s):  0.0707
Average time(s):  0.0674
Average fps:  14.8277


In [2]:
import math


def compare_points_list(tracking_dict,global_dictionary, current_list, distance_threshold, track_id, remove_from_list): ## input of form: (dictionary of (id, coordinates), the current list of points acquired, 
    add_count = 0
    tracking_dict_copy = tracking_dict.copy()                                       ##the threshold for how close points can be between frames for single object classification, and the current ID number in use (to prevent overlap))
    current_list_copy = current_list.copy()
    
    point_list = []
    print(current_list)
    
    for object_id, pt2 in tracking_dict_copy.items():
        object_exists = False
        for pt in current_list_copy:
            distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
            # Update IDs position
            if distance < distance_threshold:
                # tracking_objects2[object_id] = tracking_objects[object_id]
                tracking_dict[object_id] = pt
                object_exists = True
                point_list.append(pt)
                if pt in current_list:
                    current_list.remove(pt)
                continue
        # Remove IDs lost
        if remove_from_list:    #if this is set to True, then will remove elements from list if they are not noted in current frame. if set to false, those points will be kept. 
            if not object_exists:
                id_val = tracking_dict[object_id]
                id_val = [id_val[0],id_val[1],id_val[2],id_val[3]]
                combine_id_data = [object_id,id_val]
                global_dictionary.append(combine_id_data) ## keeping track of all id's that have been detected by the system
                tracking_dict.pop(object_id)
    
    for pt in current_list:
        add_count += 1
        tracking_dict[track_id] = pt
        track_id += 1

    return tracking_dict, current_list, track_id


In [3]:
## second iteration


## demo for object tracking

import numpy as np
import cv2
import math
from IPython.display import clear_output
import time
import csv

avg_time = []

Conf_threshold = 0.4
NMS_threshold = 0.4
COLORS = [(0, 255, 0), (0, 0, 255), (255, 0, 0),
          (255, 255, 0), (255, 0, 255), (0, 255, 255)]
caps = cv2.VideoCapture('traffic_demo.mp4')

net = cv2.dnn.readNet('yolov4-opencv-python/yolov4-tiny.weights', 'yolov4-opencv-python/yolov4-tiny.cfg')
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

model = cv2.dnn_DetectionModel(net)
model.setInputParams(size=(416, 416), scale=1/255, swapRB=True)

# Initialize count
count = 0

center_points_3prev_frame = []
center_points_2prev_frame = []
center_points_prev_frame = []
center_points_cur_frame = []

tracking_objects_2 = {}

tracking_objects = {}
tracking_objects2 = {}
track_id = 0

distance_thresh2 = 30
distance_thresh = 20 # how close objects can be between frames before counted as singular object

car_names = ['Sue', 'Billy','Bob','Alice','Jemma','Gertrude','Chad','Felicity','Grayson','Adam']

global_dictionary= []

start_time = time.time()

while True:
    _, frame = caps.read()
    
    frame = cv2.resize(frame, (800, 400), fx = 0.1, fy = 0.1,interpolation = cv2.INTER_CUBIC)
    classes, scores, boxes = model.detect(frame, Conf_threshold, NMS_threshold)    
    
    
    for box in boxes: 
        (x,y,w,h) = box
        cx = int((x + x + w) / 2)
        cy = int((y + y + h) / 2)
        # center_points_cur_frame.append((cx,cy))
        center_points_cur_frame.append((cx,cy,w,h))
        # print("Frame N:", count, x, y, w, h )
        # cv2.rectangle(frame,(x,y,),(x+w,y+h),(0,255,0),2)
        
    # print("box length" ,len(boxes))
    # cnt_pts_prv_frame = center_points_cur_frame.copy()
        
       # Only at the beginning we compare previous and current frame
#     if count <= 1:
#         for pt in center_points_cur_frame:
#             for pt2 in center_points_prev_frame:
#                 distance = math.hypot(pt2[0] - pt[0], pt2[1] - pt[1])
#                 if distance < distance_thresh:
#                     tracking_objects[track_id] = pt
#                     track_id += 1


                    
    # else:  
    print(center_points_cur_frame)
    tracking_objects, center_points_cur_frame, track_id = compare_points_list(tracking_objects,global_dictionary, center_points_cur_frame, distance_thresh, track_id, True)  ## compare previous list to current list. 
    

                
    for object_id, pt in tracking_objects.items():
        x = pt[0]
        y = pt[1]
        w = pt[2]
        h = pt[3]
        ul = (int(x - w/2), int(y - h/2))
        lr = (int(x + w/2), int(y + h/2))
        
        cv2.circle(frame, (x,y),5,(0,0,255),-1)
        cv2.rectangle(frame,ul,lr,(0,255,0),2)

        name_val = object_id % 10
        cv2.putText(frame,str(object_id),(pt[0], pt[1]-7),cv2.FONT_HERSHEY_COMPLEX,0.5,(0,0,0),1)
    
    # print(track_id, center_points_cur_frame,tracking_objects, cnt_pts_prv_frame
    # print("Tracking objects:")
    # print(tracking_objects)
                
        
        # cv2.circle(frame,pts, 2, (0,0,255),-1)
        
    #make copy of points
    clear_output(wait = True)
    print("Frame no: ", count)
    end_time = time.time()
    frame_time = end_time - start_time
    start_time = time.time()
    print("Time of frame(s): ", round(frame_time,4))
    avg_time.append(frame_time)
    average_frame_time = np.average(avg_time)
    print("Average time(s): ", round(average_frame_time,4))
    avg_fps = 1 / average_frame_time
    print("Average fps: ", round(avg_fps,4))
        
        
    cv2.imshow("Frame", frame)
    
        
        
    start_time = time.time()
    
    # tracking_objects_2 = tracking_objects.copy()
    
    # print("center current frame 2", center_points_cur_frame)
    # center_points_3prev_frame = center_points_2prev_frame.copy()
    # center_points_2prev_frame = center_points_prev_frame.copy()
    # center_points_prev_frame = center_points_cur_frame.copy() ## really only used for first two frames.
    
    # print("object lists: ",center_points_3prev_frame,center_points_2prev_frame, center_points_prev_frame, center_points_cur_frame)
        
    
    
    center_points_cur_frame = []  # if not set to [], will cause lots and lots of problems. 

    
    count += 1
              
    key = cv2.waitKey(0)
    if key == 27:
        break
        print(global_dictionary)
    
# print(global_dictionary)
fields = ['id','x','y','minor','major']
with open('global_dictionary.csv', 'w', newline = '') as f: 
    #creating  a csv writer object
    csvwriter = csv.writer(f)
    #writing the fields
    csvwriter.writerow(fields)
    # writing the data rows
    csvwriter.writerows(global_dictionary)

caps.release()
cv2.destroyAllWindows()
# print(global_dictionary)

Frame no:  9
Time of frame(s):  0.941
Average time(s):  0.9899
Average fps:  1.0102
